# VIF (Variance Inflation Factor)

El VIF es una medida que nos indica cuánto aumenta la varianza de un coeficiente de regresión debido a la colinealidad con otras variables independientes. El VIF se calcula con la siguiente fórmula:
$$
VIF_j = \frac{1}{1 - R_j^2}
$$
Esta es la fórmula estándar para calcular el Variance Inflation Factor (VIF) de la variable **j**, donde
- VIF_j es el **Variance Inflation Factor** para la variable 
- R_j^2 es el **coeficiente de determinación** de la regresión de la variable j sobre todas las otras variables independientes en el modelo.

Cuando aplicamos la formula y obtenemos el valor de VIF podemos determinar con la siguiente tabla la colinealidad de la columna
| **Rango de VIF** | **Interpretación**                                                                                                    | **Impacto en el modelo**                                                                                                                                                       |
|------------------|------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **VIF == 1**      | No hay colinealidad entre las variables.                                                                                | Las variables son completamente independientes entre sí. No hay redundancia en la información que cada una aporta al modelo. Los coeficientes estimados son confiables y estables. |
| **VIF <= 5**  | Colinealidad moderada entre algunas variables, pero no es preocupante.                                                  | Las variables tienen algo de correlación, pero no es lo suficientemente fuerte como para causar problemas graves. El modelo sigue siendo confiable y los coeficientes son estables. |
| **VIF > 5 && VIF <= 10** | Alta colinealidad. Las variables están altamente correlacionadas entre sí, lo que puede generar problemas serios.        | La alta colinealidad provoca que los coeficientes sean **inestables y difíciles de interpretar**. Esto se debe a que pequeños cambios en los datos pueden causar grandes fluctuaciones en los coeficientes. Esto **incrementa la varianza** de los coeficientes, haciendo que el modelo sea más sensible a errores de predicción. |
| **VIF >> 10**    | Muy alta colinealidad. Existe una fuerte redundancia entre las variables, lo que afecta negativamente al modelo.         | El modelo es probablemente **inestable**. Los coeficientes tienen **alta varianza** y no reflejan correctamente las relaciones entre las variables. Las predicciones y la interpretación se ven gravemente afectadas. Se recomienda eliminar o combinar variables correlacionadas. |

### Objetivo de analizar el VIF

El objetivo principal de determinar la **colinealidad** de las variables mediante el VIF es **reducir la probabilidad de generar modelos inestables**, mejorando la precisión y la interpretabilidad del modelo de regresión. Si el VIF es alto, las estimaciones de los coeficientes pueden volverse muy **sensibles a pequeños cambios** en los datos, lo que puede **afectar negativamente la calidad del modelo**.

### ¿Qué hacer cuando el VIF es alto?

1. **Eliminar variables colineales:** Si encuentras que ciertas variables están altamente correlacionadas, considera eliminar una de ellas.
2. **Combinación de variables:** Si las variables correlacionadas representan lo mismo, intenta combinarlas o crear una nueva variable que las represente a todas.
3. **Uso de regularización:** Si no puedes eliminar variables, usa técnicas de **regularización** como **Ridge** o **Lasso**, que ayudan a reducir la influencia de las variables altamente correlacionadas.
4. **Análisis de Componentes Principales (PCA):** Esta técnica puede ayudar a transformar las variables correlacionadas en un conjunto de componentes ortogonales, lo que puede mejorar la estabilidad del modelo.



## Importing libraries

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
import pandas as pd
from sklearn.preprocessing import StandardScaler

## Metodo para calcular VIF con un modelo de Regression Lineal

In [33]:
 def calculateVIF(var_predictoras_df):
     var_pred_labels = list(var_predictoras_df.columns) #Sacamos las labels para las variables
     num_var_pred = len(var_pred_labels)
    
     lr_model = LinearRegression() #Declaramos el modelo de regression
    
     result = pd.DataFrame(index = ['VIF'], columns = var_pred_labels, dtype=float)
     result = result.fillna(0)
    
     for ite in range(num_var_pred):   #En este bucle comparamos la influencia de una columna contra el resto de columnas
         x_features = var_pred_labels[:] #Sacamos todas las columnas
         y_feature = var_pred_labels[ite] #Sacamos la columna que queremos analizar
         x_features.remove(y_feature) #removemos nuestra columna target del resto
        
         x = var_predictoras_df[x_features] #Creamos los datos para x
         y = var_predictoras_df[y_feature] # Creamos los datos para el target
        
         lr_model.fit(var_predictoras_df[x_features], var_predictoras_df[y_feature]) #Entrenamos el modelo
        
         result[y_feature] = 1/(1 - lr_model.score(var_predictoras_df[x_features], var_predictoras_df[y_feature])) #Realizamos la formula para calcular el VIF de esa columna
    
     return result.T.sort_values(by="VIF", ascending=False)

## Metodo para calcular VIF con un modelo Lasso

In [23]:

def calculateVIF_lasso(var_predictoras_df):
    var_pred_labels = list(var_predictoras_df.columns)
    num_var_pred = len(var_pred_labels)
    
    result = pd.DataFrame(index = ['VIF'], columns = var_pred_labels)
    
    for ite in range(num_var_pred):
        x_features = var_pred_labels[:]
        y_feature = var_pred_labels[ite]
        x_features.remove(y_feature)

        x = var_predictoras_df[x_features]
        y = var_predictoras_df[y_feature]

        # Escalamos porque Lasso es sensible a la escala
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x)
        y_scaled = StandardScaler().fit_transform(y.values.reshape(-1, 1)).ravel()

        # Lasso con validación cruzada para encontrar el mejor alpha
        lasso = LassoCV(cv=5, random_state=42, max_iter=10000)
        lasso.fit(x_scaled, y_scaled)

        r2 = lasso.score(x_scaled, y_scaled)
        vif_value = 1 / (1 - r2) if r2 < 1 else float('inf')
        result[y_feature] = vif_value

    return result.T.sort_values(by="VIF", ascending=False)


## Metodo para calcular VIF con un modelo Ridge

In [24]:
def calculateVIF_ridge(var_predictoras_df):
    var_pred_labels = list(var_predictoras_df.columns)
    num_var_pred = len(var_pred_labels)
    
    result = pd.DataFrame(index = ['VIF'], columns = var_pred_labels)
    
    for ite in range(num_var_pred):
        x_features = var_pred_labels[:]
        y_feature = var_pred_labels[ite]
        x_features.remove(y_feature)

        x = var_predictoras_df[x_features]
        y = var_predictoras_df[y_feature]

        # Escalamos porque Ridge es sensible a la escala
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(x)
        y_scaled = StandardScaler().fit_transform(y.values.reshape(-1, 1)).ravel()

        # Ridge con validación cruzada para encontrar el mejor alpha
        ridge = RidgeCV(cv=5)
        ridge.fit(x_scaled, y_scaled)

        r2 = ridge.score(x_scaled, y_scaled)
        vif_value = 1 / (1 - r2) if r2 < 1 else float('inf')
        result[y_feature] = vif_value

    return result.T.sort_values(by="VIF", ascending=False)

## Importamos los datos

In [25]:
direction_train = "../Train_knight.csv"
data = pd.read_csv(direction_train)
data["knight"] = data["knight"].map({"Jedi": 1, "Sith": 0})

## Procesamos los datos

In [26]:
X = data.drop("knight", axis=1)

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
X_normalized = pd.DataFrame(X_normalized, columns=X.columns)

## Calculamos VIF

In [48]:
vif_reg = calculateVIF(X)
vif_lasso = calculateVIF_lasso(X)
vif_ridge = calculateVIF_ridge(X)

vif_reg.columns = ['VIF_Reg']
vif_lasso.columns = ['VIF_Lasso']
vif_ridge.columns = ['VIF_Ridge']

vif_combined = pd.concat([vif_lasso, vif_ridge, vif_reg], axis=1)

print (vif_combined.sort_values("VIF_Lasso",ascending=False))

/var/folders/cz/_60q6gpj4kxc5qmxykyvhyfr0000gn/T/ipykernel_20771/1415708530.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = result.fillna(0)


                 VIF_Lasso    VIF_Ridge      VIF_Reg
Strength       2513.809675  4178.462950  4489.653450
Sensitivity    2139.547229  4134.233130  4367.233204
Recovery        487.232037   618.184037   782.388684
Power           302.516910   405.143815   411.965245
Sprint          282.855384   353.134389   359.129874
Stims           270.665297   330.881047   346.022133
Slash            65.553042    78.346997    80.754427
Pull             63.646589    72.428095    73.911907
Awareness        62.217280    67.872474    69.852627
Prescience       53.845545    54.186763    54.712644
Lightsaber       43.433526    46.745628    47.550260
Dexterity        39.575415    45.748316    54.126657
Attunement       32.342326    28.994658    33.904833
Empowered        31.047759    30.001861    36.224859
Delay            23.527120    30.418466    35.987392
Friendship       17.189769    14.808709    19.040267
Grasping         15.992533    17.496006    19.758797
Evade            15.756696    16.399529    17.

## Elegimos Aquellas columnas cuyo VIF es < 5

In [47]:
def select_columns_vif_lower_5(vif_data):
    vif_5 = vif_data[(vif_data < 5)]
    vif_5.dropna(inplace = True)
    print ("-",vif_5.columns[0],"-")
    print (vif_5)
    print()
    
select_columns_vif_lower_5(vif_lasso)
select_columns_vif_lower_5(vif_ridge)
select_columns_vif_lower_5(vif_reg)

- VIF_Lasso -
            VIF_Lasso
Deflection   4.221709
Reactivity   3.953632
Push         3.651255
Survival     2.703836

- VIF_Ridge -
            VIF_Ridge
Deflection   4.234272
Push         4.051862
Reactivity   4.040854
Survival     3.363717

- VIF_Reg -
             VIF_Reg
Deflection  4.550873
Reactivity  4.074260
Push        4.065017
Survival    3.630928

